In [1]:

"""
创造一个多人表演的搞笑视频剧本。

主要通过多智能体协作：multi-agent system.

"""


import os, json
from langchain_openai import ChatOpenAI

from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

from typing import TypedDict, Annotated, List, Optional

from langgraph.graph import StateGraph, END, START






In [2]:


prompt_scene = "你是一个资深的搞笑视频剧本的创作者，设计一段搞笑剧本，题材是：{topic}, 根据题材设计一个搞笑剧本的背景和角色，但不要涉及搞笑内容，尽可能简短，不超过60字，角色设计不超过5个。"

prompt_roles = "识别搞笑剧本中的角色，并以列表的形式返回,角色名称必须完全匹配。"

prompt_select_speaker = """根据搞笑的题材、已有对话内容，从下面对话角色中选择一个角色进行发言，如果没有对话内容，随机选择一个角色发言；如果对话内容，结束返回END；选择的角色不要与当前角色相同。
搞笑题材：{topic}
已有对话内容：{dialog_messages}
当前对话角色：{current_role}
可以供选择的角色：{roles}
选择发言的角色：
"""

prompt_speaker_message = """现在在编写搞笑剧本，题材：{topic}, 你现在是{speaker}, 根据已有的对话和场景，讲述你自己的发言，要求你讲话内容：
1. 讲话内容保持你前后一致的说话风格。
2. 讲话内容能够推动搞笑剧情的发展。
3. 不要重复之前的台词。


已有的对话内容：
{dialog_messages}
"""



In [3]:


class State(TypedDict):
    topic: Annotated[str, "搞笑创作的题材"]
    next_speaker: Annotated[str, "下一个发言的角色"]
    history_messages: Annotated[List[str], "搞笑创作内容的历史对话记录"]
    # current_response: Annotated[str, "当前角色发言的内容"]

    current_speaker: Annotated[str, "当前发言的角色"]
    dialog_lefts: Annotated[int,"剩余的最大对话轮次"]
    roles: Annotated[List[str], "搞笑创作内容中的发言角色名字"]
    results: Annotated[str, "最终完整的搞笑对话内容"]


model = "Qwen2.5-7B-Instruct-AWQ"
# model = "internlm2_5-7b-chat"

llm = ChatOpenAI(api_key="EMPTY", base_url="http://localhost:8000/v1", model=model,
                temperature=0.7,
                max_tokens=None)


In [4]:
from pydantic import BaseModel, Field


# 创建搞笑的背景


# 创建角色

class TopicRole(BaseModel):
    background: str = Field(description="搞笑题材的背景")
    topic_roles: List[str] = Field(description="搞笑题材中的角色列表,每个角色格式：姓名(角色身份)")

topic_role_llm_with_structured_output = llm.with_structured_output(TopicRole)


def create_role_agent(state: State) -> State:
    print("-----------------create_role_agent--------------------------")
    print(state)
    topic = state["topic"]
    prompt = PromptTemplate.from_template(prompt_scene)
    
    chain = prompt | topic_role_llm_with_structured_output

    response = chain.invoke({"topic": topic})

    return {"topic": topic, "roles": response.topic_roles, "history_messages": [""], "dialog_lefts": state["dialog_lefts"]}

class NextSpeaker(BaseModel):
    next_speaker: str = Field(description="从所有角色中选择下一个角色进行发言，选择的角色不要与当前角色相同;入股当前角色为空，可以从所有角色中随机选择一个角色")

next_speaker_llm_with_structured_output = llm.with_structured_output(NextSpeaker)

def select_speaker(state: State) -> State:
    print("-----------------select_speaker--------------------------")
    print(state)

    roles = state["roles"]
    topic = state["topic"]
    if "history_messages" not in state:
        dialog_messages = [""]
    dialog_messages = state["history_messages"]
    dialog_text = "\n".join(dialog_messages)
    if "current_speaker" not in state:
        current_role = ""
    else:
        current_role = state["current_speaker"]
    # print({"topic": topic, "dialog_messages": dialog_text,"current_role": current_role,"roles": roles})
    # print(prompt_select_speaker.format({"topic": topic, 
    #                                     "dialog_messages": dialog_text,
    #                                     "current_role": current_role, 
    #                                     "roles": roles}))
    prompt = PromptTemplate.from_template(prompt_select_speaker)
    chain = prompt | next_speaker_llm_with_structured_output
    # next_speaker = next_speaker_llm_with_structured_output(prompt_select_speaker.format(topic=topic,   
    #                                                                                      dialog_messages=dialog_text,
    #                                                                                      current_role= current_role,
    #                                                                                      roles=roles))
    
    # next_speaker = chain.invoke(dict(topic=topic,dialog_messages=dialog_text,current_role= current_role,roles=roles))
    
    # next_speaker = next_speaker.next_speaker
    max_select_num = 10
    while max_select_num > 0:
        next_speaker = chain.invoke(dict(topic=topic,dialog_messages=dialog_text,current_role= current_role,roles=roles))
        next_speaker = next_speaker.next_speaker
        if next_speaker in roles+["END"] and next_speaker != current_role:
            break
        else:
            max_select_num -= 1
            
    
        
    
    if next_speaker == "END":
        return {"dialog_lefts": 0}

    return {"next_speaker": next_speaker}


# 对话处理

def handle_dialog(state: State):
    print("-----------------handle_dialog--------------------------")
    print(state)
    dialog_lefts = state["dialog_lefts"]
    next_speaker = state["next_speaker"]
    roles = state["roles"]
    topic = state["topic"]
    if "history_messages" not in state:
        dialog_messages = ""
        history_messages = [""]
    else:
        dialog_messages = "\n".join(state["history_messages"])
        history_messages = state["history_messages"]
    prompt = prompt_speaker_message.format(topic=topic, speaker=next_speaker, 
                                           dialog_messages=dialog_messages)
    
    speaker_response = llm(prompt).content
    

    # 更新状态
    state["dialog_lefts"] = state["dialog_lefts"] - 1
    state["current_speaker"] = next_speaker
    state["history_messages"] = history_messages + [next_speaker + ":" + speaker_response]
    state["results"] = dialog_messages + "\n" + next_speaker + ":" + speaker_response
    return dict(dialog_lefts=state["dialog_lefts"] - 1, current_speaker=next_speaker,
               history_messages=history_messages + [next_speaker + ":" + speaker_response],
               results=dialog_messages + "\n" + next_speaker + ":" + speaker_response)


def check_end(state: State):

    if state["dialog_lefts"] <= 0:
        return END
    else:
        return "continue"



In [5]:

# 定义工作流

workflow = StateGraph(State)


workflow.add_node("create_role_agent", create_role_agent)
workflow.add_node("select_speaker", select_speaker)
workflow.add_node("handle_dialog", handle_dialog)
# workflow.add_node("check_end", "check_end")

workflow.add_edge("create_role_agent", "select_speaker")
workflow.add_edge("select_speaker", "handle_dialog")
# workflow.add_edge("handle_dialog", "check_end")

workflow.add_conditional_edges("handle_dialog", check_end, {END: END, "continue": "select_speaker"})

workflow.set_entry_point("create_role_agent")


app = workflow.compile()


In [14]:
# 测试内容


inputs = {"topic": "美国选举", "dialog_lefts": 5}
response = app.invoke(inputs)


-----------------create_role_agent--------------------------
{'topic': '美国选举', 'dialog_lefts': 5}
-----------------select_speaker--------------------------
{'topic': '美国选举', 'history_messages': [''], 'dialog_lefts': 5, 'roles': ['约翰', '玛丽', '彼得', '苏珊', '汤姆']}
-----------------handle_dialog--------------------------
{'topic': '美国选举', 'next_speaker': '汤姆', 'history_messages': [''], 'dialog_lefts': 5, 'roles': ['约翰', '玛丽', '彼得', '苏珊', '汤姆']}
-----------------select_speaker--------------------------
{'topic': '美国选举', 'next_speaker': '汤姆', 'history_messages': ['', '汤姆:汤姆：（拿起麦克风，笑容满面）大家好！今天我在这里，既是来给这个美国选举的舞台增添点幽默，也是为了尝试理解那些让民众难以理解的选举策略。你们知道吗？在这次选举中，我们有两位候选人，一个说“我会让你们看到星星”，另一个则说“我只会让你看到我的笑脸”。这简直就像在玩猜谜游戏！\n\n（观众们发出笑声）\n\n汤姆：（继续）但是，别担心，我汤姆-幽默大师，会帮助大家理解这个谜题。比如说，那位说“我只会让你看到我的笑脸”的候选人，我认为他可能是在暗示他有一个神奇的面具，能够让他在任何情况下都保持微笑。这可真是个天才的想法啊！（笑声）\n\n（一位观众举手提问）\n\n观众：汤姆，你认为谁会赢得这次选举？\n\n汤姆：（耸耸肩）嗯，我是一个喜剧演员，不是政治分析师。但我可以告诉你，我的预测是：如果选手们在下一轮辩论中都选择穿上了超级英雄的服装，那么我们可能会迎来一位超级英雄总统。毕竟，谁不喜欢超级英雄呢？（笑声和欢呼）\n\n（汤姆从口袋里掏出一张纸，上面写

In [16]:
response["results"]

'\n汤姆:汤姆：（拿起麦克风，笑容满面）大家好！今天我在这里，既是来给这个美国选举的舞台增添点幽默，也是为了尝试理解那些让民众难以理解的选举策略。你们知道吗？在这次选举中，我们有两位候选人，一个说“我会让你们看到星星”，另一个则说“我只会让你看到我的笑脸”。这简直就像在玩猜谜游戏！\n\n（观众们发出笑声）\n\n汤姆：（继续）但是，别担心，我汤姆-幽默大师，会帮助大家理解这个谜题。比如说，那位说“我只会让你看到我的笑脸”的候选人，我认为他可能是在暗示他有一个神奇的面具，能够让他在任何情况下都保持微笑。这可真是个天才的想法啊！（笑声）\n\n（一位观众举手提问）\n\n观众：汤姆，你认为谁会赢得这次选举？\n\n汤姆：（耸耸肩）嗯，我是一个喜剧演员，不是政治分析师。但我可以告诉你，我的预测是：如果选手们在下一轮辩论中都选择穿上了超级英雄的服装，那么我们可能会迎来一位超级英雄总统。毕竟，谁不喜欢超级英雄呢？（笑声和欢呼）\n\n（汤姆从口袋里掏出一张纸，上面写着他的新点子）\n\n汤姆：（笑）另外，我还想提议在选举中增加一项规则：任何候选人做出承诺时，必须首先列出一个完成这个承诺的详细步骤，然后还需要附上一份他们完成这些步骤的计划书。这样我们就可以看看他们是否真的有解决国家问题的能力，而不是仅仅在口号上做文章。\n\n（汤姆的提议引发了观众的热烈讨论和笑声）\n\n汤姆：（微笑）好了，各位，让我们用笑声来结束今天的讨论。记住，无论选举结果如何，请用笑来面对挑战，因为它总是能够让我们在困难时刻找到希望。下次我们再一起笑对人生吧！（挥手向观众致意）\n\n（观众们热烈鼓掌，汤姆满意地离开了舞台）\n玛丽:玛丽：（走上舞台，笑容可掬地向观众鞠躬）嗨，大家好！我是玛丽，一个热爱喜剧的人。在这个充满激烈竞争的选举舞台上，我有时会想，如果我们把选举变成一场表演，是不是就能吸引更多的观众呢？\n\n（观众发出笑声，玛丽继续）\n\n玛丽：（夸张地）我不只是在开玩笑。想象一下，如果每位候选人首先需要通过一场“才艺展示”来吸引选民，比如唱歌、跳舞、甚至是讲笑话，这岂不是能让我们见识到他们的多才多艺？\n\n（观众大笑，玛丽继续）\n\n玛丽：（眼睛一亮）或者我们可以设立一个环节，让候选人穿上他们“梦想中的”总统服装。这样一来，我们就能看到他们如何以不同的角色去处理国家问题了。比如，一

In [8]:

"""测试prompt_scene下的输出结果"""
prompt = PromptTemplate.from_template(prompt_scene)
    
chain = prompt | topic_role_llm_with_structured_output

response = chain.invoke({"topic": "美国选举"})

# print({"roles": response.topic_roles})
print(response)


"""测试角色选择情况"""
print(llm("介绍你自己。"))

background='在美国紧张的选举周期中，社交媒体上充斥着各种政治广告。' topic_roles=['疯狂的民主党支持者', '死忠的共和党支持者', '中立的观察者', '假新闻制造者', '社交媒体平台管理员']
content='我是来自阿里云的超大规模语言模型，我叫通义千问。我可以回答各种问题、创作文字，比如写故事、写公文、写邮件、写剧本等等，还能表达观点、玩梗，并且监督和纠正错误的表述。我可以帮人们创作出高质量的文案，提高他们的工作效率。我会持续学习新的知识，保持更新。我可以做很多事情，帮助人们在不同的场景中获得便利和乐趣。如果您有任何问题或需要帮助，请随时告诉我，我会尽力提供支持。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 108, 'prompt_tokens': 22, 'total_tokens': 130, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'Qwen2.5-7B-Instruct-AWQ', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-638c4adc-628e-49e3-9a5a-1d36b795a588-0' usage_metadata={'input_tokens': 22, 'output_tokens': 108, 'total_tokens': 130, 'input_token_details': {}, 'output_token_details': {}}


In [9]:



prompt_select_speaker = """根据搞笑的题材、已有对话内容，从下面对话角色中选择一个角色进行发言，如果没有对话内容，随机选择一个角色发言；如果对话内容，结束返回END；选择的角色不要与当前角色相同。
搞笑题材：{topic}
已有对话内容：{dialog_messages}
当前对话角色：{current_role}
可以供选择的角色：{roles}
选择发言的角色：
"""


prompt_select_speaker.format(topic= "美国", dialog_messages= [""], current_role="", roles="")


"根据搞笑的题材、已有对话内容，从下面对话角色中选择一个角色进行发言，如果没有对话内容，随机选择一个角色发言；如果对话内容，结束返回END；选择的角色不要与当前角色相同。\n搞笑题材：美国\n已有对话内容：['']\n当前对话角色：\n可以供选择的角色：\n选择发言的角色：\n"